### RCNN与SPPnet的不足
- 1. 训练的pipeline步骤多,
- 2. 训练代价高, 占用的空间与时间多
- 3. 推理过程慢, 难以用于实际应用


### Fast RCNN
- 1. 端到端训练, 多任务损失
- 2. 训练能更新整个网络的参数
- 3. 无需缓存特征


### 网络结构
输入为原始图片及RoI

先将原图经过卷积网络得到特征图,

再通过RoI pooling(单个的SPP层)从特征图提取RoI特征向量


SPPnet无法更新SPP层下的权重,
由于RoI取自整张图片,感受野大,梯度在不同图片的RoI中的传播十分低效,

### 多任务损失
$L(p,u,t^u,v) = L_{cls}(p,u)+\lambda [u \geq 1]L_{loc}(t^u,v)$

$类别损失L_{cls}(p,u)=-log p_u$

boundingbox损失
\begin{equation}
L_{loc}(t^u,v)=\sum _{i\in {x,y,w,h}}smooth_{L_1}(t_i^u-v_i)
\end{equation}

$
smooth_{L_1}(x)=
\left\{
\begin{aligned}
   0.5x_2& & if |x|<1  \\
   |x|-0.5& & otherwise,
 \end{aligned}
 \right.
$
$文章中的L_1损失对轮廓的不那么敏感,相较于RCNN与SPPnet中的L_2损失； 当回归的目标值没有限制的时候, L_2u损失对学习率要特别小心, 容易梯度爆炸$

### mini-batch sampling
全排列数据集,随机取其中一对, RoI大小为128,

其中四分之一正样本RoI,IoU大于0.5,

剩下的为背景,IoU在0.1至0.5之间

### SVD加快检测
对整个分类任务,卷积上花的时间比FC层要多,

但RoI数量的检测上,FC层上花费的时间占了近整个前匮网络的一半,
一个参数较多的FC层能在SVD后大大的加速

$W\approx U\Sigma_tV^T$


能将参数量有uv降至t(u+v)

$具体做法是将一层FC层换成两层,第一层权重为\Sigma_tV^T, 第二层权重为U$

当RoI数量很大时能加速很多